## Usage of python script

In [3]:
%%time
import reader

reader.query('Mathematics', limit=1000)

Dumped 100 pages
Dumped 200 pages
Dumped 300 pages
Dumped 400 pages
Dumped 500 pages
Dumped 600 pages
Dumped 700 pages
Dumped 800 pages
Dumped 900 pages
Dumped 1000 pages
Wall time: 32 s


## Implementation (Debug purposes)

In [17]:
from pathlib import Path
import pywikibot
from pywikibot import pagegenerators
import json
import mwparserfromhell as mwp

def _clean(wiki_text):
    wikicode = mwp.parse(wiki_text)
    return wikicode.strip_code()

def _dump(path, data):
    with open(path, 'w', encoding='utf8') as outfile:  
        json.dump(data, outfile, indent=2, ensure_ascii=False)

def query(request, batch_size=100):
    requests_base = Path('../requests')
    requests_path = requests_base / request
 
    if requests_path.exists():
        return

    requests_path.mkdir(parents=True)
    
    site = pywikibot.Site()
    category = pywikibot.Category(site, request)
    pages = category.articles(namespaces=[0], #type of entities to query, 0 = page
                              recurse=True, # also query all subpages
                              content = True) # preloaod pages
    
    count = 0
    pages_key = 'pages'
    data = { pages_key: [] }
    for p in pages:
        count += 1
        data[pages_key].append({
            'title': p.title(),
            'url': p.full_url(),
            'text': _clean(p.text),
        })
        
        if count % batch_size == 0:
            _dump(requests_path / (str(count) + '.json'), data)
            data = { pages_key: [] }
            print('Dumped {} pages'.format(count))
            
        if count == 1000:
            break
            
    if len(data[pages_key]):
        _dump(requests_path / (str(count) + '.json'), data)